이거는 파일 개수가 1개 다를때!!!  
원래 T1이 189개, FLAIR가 188개라서  
NAn 처리해주고 했는데...사실 데이터 잘 못받아서  
개수가 189개로 동일했음...

In [ ]:
import os
import numpy as np
import pandas as pd
import shutil
import pydicom
from tqdm import tqdm_notebook

In [ ]:
# 1) 교수님께서 주신 환자 엑셀 리스트 불러오기!!

training_set_dir = 'D:\\gisung\\task_2'

training_set = pd.read_excel(os.path.join(training_set_dir, '20200622_Training_set.xlxs'))
pt_origin_list = training_set['original_MRN']
pt_origin_list

In [ ]:
# 2) 폴더 돌면서 시퀀스 받아오기

srcDir = 'Z\\Metastasis\\T1_FLAIR\\FLAIR'

check_seq = set()
# seq 중복되는거 피하려고!!
pt_seq_dic = dict()
error_list = []

for path, dirs, files in tqdm_notebook( os.walk(srcDir) ):
    if files:
        for file in tqdm_notebook(files, total = len(files)):
            pt = path.split(os.path.sep)[-1].split('_')[0]
            pt = int(pt)
            
            if pt not in pt_seq_dic:
                # pt 키가 없으면 생성 + set로 넣어서 중복되는 거 제외 시킬거임.
                
                pt_seq_dic[pt] = set()
                
            # 다이콤 읽어와서 seq 저장!!
            dcmData = pydicom.dcmread(os.path.join(path, file))
            
            Pt_id = dcmData[0x00100020].value
            seq = dcmData[0x008103e].value
            
#             print(type(Pt_id)) 문자열로 뽑힘
            
            Pt_id = int(Pt_id) # 위에서 pt를 정수형으로 맞춰줬고 엑셀에서 뽑은것도 정수형이라서
        
            if pt != Pt_id:
                error_list.append(os.path.join(path,file))
                continue
                # 만약 다이콤 파일의 환자 id랑 폴더에 있는 id랑 다르면 error_list에 넣어줌!!
                
            pt_seq_dic[Pt_id].add(seq)
            
print(pt_seq_dic)

## 중간에 오류 뜬 이유 보니까...jpg로 1개 잘못 저장함...
## 그리고 시간 오래걸림!!3D_FLAIR는!!

In [ ]:
print(error_list)
## error_list 비어 있는거 확인 :)
## 그 말은 다이콤 id가 알맞은 환자 id 폴더에 들어있다는 의미

In [ ]:
# pt_df = pd.DataFrame(pt_origin_list, columns = ['pt_origin_id'])
# 이거는 오류 뜸....왜지..?

# 3) 기존 excel에 있는 환자 순서대로 목록 정리
pt_df = pd.DataFrame(pt_origin_list)
pt_df

In [ ]:
# 4) 우리가 돌면서 받아온 환자 seq리스트

pt_seq_df = pd.DataFrame(pt_seq_dic)
pt_seq_df = pt_seq_df.T
pt_seq_df = pt_seq_df.rename(columns = { 0 : 'pt_seq'})
pt_seq_df

In [ ]:
# 인덱스도 환자 번호라서 column으로 빼주기
pt_seq_df = pt_seq_df.reset_index()
pt_seq_df = pt_seq_df.rename(columns = { 'index' : 'pt_id'})
pt_seq_df

# 행 수가 하나 부족한 것을 볼 수가 있다.

In [ ]:
# 5) 환자 id 기준으로 합치기!!
## 기존 original 순서에다가 합치기

pt_seq_final = pd.merge(pt_df, pt_seq_df.rename(columns = {'pt_id' : 'original_MRN'}), how = 'left', on = 'original_MRN' )
# 기존 엑셀에 있는 환자 id 순인 pt_df를 기준으로 합침
# 만약 컬럼 이름이 같으면 상관이 없는데 다르면은 left_on, right_on으로 기준 잡아줘야 함.
# 근데 이름 다르면 그게 두개가 들어감...그래서 한번에 해주려면 반대쪽 column명을 바꿔서 통일해줘야 함.
# 여기서 rename하고 원본 변수에 할당 하지 않으면 원본값은 변하지 않음.
# 대신에 이름을 바꾸었으니 on 한번만 해주면 됨.
pt_seq_final

기존 original은 189개의 행이고 우리가 붙이려는거는 188개의 행이지만

그냥 T1과정과 동일하게 왼쪽 기준으로 해주고 key name을 동일하게 해주면

왼쪽 key 기준으로 들어가고 없는거는 Nan으로 들어가게 됨.

==============================

In [ ]:
# 환자 seqlist에 시퀀스 1개만 들어가야 하는데 혹시 다른거 들어가 있는지는 않은지 check
num = 0

for i in pt_seq_final['pt_seq']:
    
    if len(list(i)) == 1 :
        pt_seq_final['pt_seq'][num] = list(i)[0]
        
    else:
        break
        
    num += 1
    
print(num)
## 만약 다 값이 1개 이면은 num값은 189, 즉 마지막 인덱스까지 다 돌고 1을 더한 189가 됨.

T1과 동일하게 했는데...

Nan은 float값이라서 len 계산이 불가...

===========================

그래서 nan을 df.fillna(값) 으로 바꾸고 그거에 대해 검사

값은 숫자도 되고 문자도 가능~~

In [ ]:
pt_seq_final = pt_seq_final.fillna('x')

In [ ]:
# 환자 seqlist에 시퀀스 1개만 들어가야 하는데 혹시 다른거 들어가 있는지는 않은지 check
num = 0

for i in pt_seq_final['pt_seq']:
    
    if i == x:
        num += 1
        continue
        # 결측치를 x로 반환하고 이를 x로 처리!!
        # 이미 뭐 처리를 해서 건드리지 않고 
        # 인덱스만 올리고 for문 계속
    
    elif len(list(i)) == 1 :
        pt_seq_final['pt_seq'][num] = list(i)[0]
        num += 1
        
    else:
        break
        
    
    
print(num)
## 만약 다 값이 1개 이면은 num값은 189, 즉 마지막 인덱스까지 다 돌고 1을 더한 189가 됨.

In [ ]:
# 6) T1 시퀀스 추출

pt_seq_final.to_csv( '{}'.format(os.path.join('Z:\\Metastasis\\T1_FLAIR', 'FLAIR_seq.csv')), index = False)